In [1]:
import numpy as np

class TicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3), dtype=int)  # 0 = empty, 1 = player, -1 = AI

    def reset(self):
        self.board = np.zeros((3, 3), dtype=int)

    def make_move(self, row, col, player):
        if self.board[row, col] == 0:
            self.board[row, col] = player
            return True
        return False

    def check_winner(self):
        # Check rows, columns, and diagonals for a winner
        for i in range(3):
            if abs(sum(self.board[i, :])) == 3 or abs(sum(self.board[:, i])) == 3:
                return np.sign(sum(self.board[i, :]))  # Winner is player or AI
        if abs(self.board.trace()) == 3 or abs(np.fliplr(self.board).trace()) == 3:
            return np.sign(self.board.trace())
        if not (self.board == 0).any():  # Draw if board is full
            return 0
        return None  # Game is still ongoing

    def print_board(self):
        for row in self.board:
            print(" ".join([str(cell) if cell != 0 else "." for cell in row]))
        print()

# Initialize and display the game board
game = TicTacToe()
game.print_board()


. . .
. . .
. . .



In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

def create_model():
    model = Sequential()
    model.add(Dense(128, input_dim=9, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(9, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
    return model

model = create_model()
model.summary()


2024-11-10 20:45:50.495071: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-10 20:45:50.727969: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 20:45:54.444020: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-10 20:45:54.595821: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not ope

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1280      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 9)                 297       
                                                                 
Total params: 11,913
Trainable params: 11,913
Non-trainable params: 0
_________________________________________________________________


In [3]:
def generate_training_data():
    data = []
    for _ in range(1000):  # Generate a dataset of 1000 games
        game = TicTacToe()
        states = []
        moves = []
        while True:
            current_state = game.board.flatten()
            available_moves = list(zip(*np.where(game.board == 0)))
            if not available_moves:
                break
            move = available_moves[np.random.choice(len(available_moves))]
            states.append(current_state)
            moves.append(move)
            game.make_move(*move, 1 if len(states) % 2 == 1 else -1)  # Alternate between player and AI
            if game.check_winner() is not None:
                break
        data.append((states, moves, game.check_winner()))
    return data

training_data = generate_training_data()


In [4]:
def preprocess_data(training_data):
    X, y = [], []
    for states, moves, result in training_data:
        for i in range(len(states)):
            X.append(states[i])
            target = np.zeros(9)
            move = moves[i][0] * 3 + moves[i][1]
            target[move] = 1 if result == 1 else -1  # Reward win moves, penalize loss moves
            y.append(target)
    return np.array(X), np.array(y)

X, y = preprocess_data(training_data)
model.fit(X, y, epochs=50, batch_size=32)


Epoch 1/50
 24/240 [==>...........................] - ETA: 0s - loss: 0.1138 

2024-11-10 20:46:41.044410: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


240/240 [==============================] - 2s 5ms/step - loss: 0.1115
Epoch 2/50
240/240 [==============================] - 1s 5ms/step - loss: 0.1105
Epoch 3/50
240/240 [==============================] - 1s 5ms/step - loss: 0.1101
Epoch 4/50
240/240 [==============================] - 1s 5ms/step - loss: 0.1096
Epoch 5/50
240/240 [==============================] - 1s 5ms/step - loss: 0.1092
Epoch 6/50
240/240 [==============================] - 1s 5ms/step - loss: 0.1087
Epoch 7/50
240/240 [==============================] - 1s 5ms/step - loss: 0.1083
Epoch 8/50
240/240 [==============================] - 1s 5ms/step - loss: 0.1078
Epoch 9/50
240/240 [==============================] - 1s 5ms/step - loss: 0.1074
Epoch 10/50
240/240 [==============================] - 1s 5ms/step - loss: 0.1069
Epoch 11/50
240/240 [==============================] - 1s 5ms/step - loss: 0.1065
Epoch 12/50
240/240 [==============================] - 1s 5ms/step - loss: 0.1061
Epoch 13/50
240/240 [===============

In [6]:
def model_move(game, model):
    state = game.board.flatten().reshape(1, -1)
    predictions = model.predict(state)[0]
    for i in np.argsort(predictions)[::-1]:
        row, col = divmod(i, 3)
        if game.board[row, col] == 0:
            return (row, col)
    return None  # No valid moves

# Play a game against the AI
game = TicTacToe()
while True:
    game.print_board()
    row, col = map(int, input("Enter your move (row col): ").split())
    game.make_move(row, col, 1)
    if game.check_winner() is not None:
        break

    ai_move = model_move(game, model)
    game.make_move(ai_move[0], ai_move[1], -1)
    if game.check_winner() is not None:
        break

winner = game.check_winner()
if winner == 1:
    print("You won!")
elif winner == -1:
    print("AI won!")
else:
    print("It's a draw!")


. . .
. . .
. . .

1/1 [==============================] - 0s 22ms/step
. 1 .
. . .
. -1 .

1/1 [==============================] - 0s 18ms/step
. 1 1
. . .
. -1 -1

1/1 [==============================] - 0s 20ms/step
. 1 1
. -1 .
1 -1 -1

You won!
